In [1]:
from nltk.util import ngrams
import nltk
from collections import Counter
from collections import defaultdict

In [2]:
preprocess=lambda line: line.replace('.',' ').replace(',',' ').replace(':',' ').replace(';',' ').replace('!',' ').replace('?',' ').replace('(',' ').replace(')',' ').replace("-"," ").replace(" '",' ').replace("' ",' ').replace('"',' ').replace("_"," ").replace('[',' ').replace(']'," ").replace('{',' ').replace('}'," ").replace('*',' ')   

In [3]:
def getTokens(file):
    tokens=[]
    #with open('/home/meghana/nltk_data/corpora/gutenberg/training_corpus.txt') as f:
    with open(file) as f:
        for line in f:
            if not line.isspace():
                line=line.lower()
                line=preprocess(line)
            items=line.split()
            tokens+=items
    return tokens

In [4]:
def find_perplexity(prob_table,test_quads,n):
    pp=1.0
    for quad in test_quads:
        try:
            pp=pp*((1/float(prob_table[quad[0:3]][quad[3]]))**(1/n))
        except:
            pass
    print(pp)

In [5]:
train_tokens=getTokens('training.txt')
test_tokens=getTokens('test_set.txt')
held_tokens=getTokens('held_set.txt')
print("        Train   Test   Held-out")
print("Total ",len(train_tokens),len(test_tokens),len(held_tokens))
train_tokenSet=set(train_tokens)
test_tokenSet=set(test_tokens)
held_tokenSet=set(held_tokens)
print("Unique  ",len(train_tokenSet),len(test_tokenSet)," ",len(held_tokenSet))
# 1891868 103419 104594
# 35487 12831 11647

        Train   Test   Held-out
Total  1889452 104950 105479
Unique   41617 11188   11253


In [6]:
import operator
unigrams=list(ngrams(train_tokens,1))
bigrams=list(ngrams(train_tokens,2))
trigrams=list(ngrams(train_tokens,3))
quadgrams=list(ngrams(train_tokens,4))
pentagrams=list(ngrams(train_tokens,5))
V=len(set(pentagrams))

held_pentagrams=list(ngrams(held_tokens,5))
held_pentagramSet=set(held_pentagrams)

test_pentagrams=list(ngrams(test_tokens,5))
test_pentagramSet=set(test_pentagrams)

unigram_table=Counter(train_tokens)
bigram_table=Counter(bigrams)#stores the frequencies of each bigram similarly trigrams and quadgrams
trigram_table=Counter(trigrams)
quadgram_table=Counter(quadgrams)
pentagram_table=Counter(pentagrams)

In [8]:
for key in pentagram_table:
    quad=key[0:4]
    pentagram_table[key]=((pentagram_table[key])/(quadgram_table[quad]))
                         
penta_prob_table=defaultdict(dict)

for penta in pentagram_table:
    p=pentagram_table[penta]
    quad=penta[0:4]
    token=penta[4]
    penta_prob_table[quad][token]=p

In [13]:
def find_score(penta_prob_table,test_pentagramSet):
    score=0
    print(len(test_pentagramSet))
    for penta in test_pentagramSet:
        
        quad=penta[0:4]
        token=penta[4]
        result=list(penta_prob_table[quad])
        #print(result)
        try:
            if token==result[0]:
                score+=1
        except:
            pass
    print(score)

In [14]:
find_score(penta_prob_table,test_pentagramSet)

104336
3585


In [17]:
freq_table=[]
pentagram_table=Counter(pentagrams)#stores the frequencies of each quadgram
quadgram_table=Counter(pentagrams)
#stores the frequencies of (counts of quadgrams)eg.N1=150 => no of quadgrams with count 1 are 150
for penta in pentagram_table:
    freq_table.append(pentagram_table[penta])
    
freq_table=Counter(freq_table)
freq_table=dict(sorted(freq_table.items(), key=operator.itemgetter(0), reverse=True))
#print(freq_table)

#history_dict contains no. of different trigrams a token follows
#future_dict contains no. different tokens a trigram follows
history_dict=defaultdict(int)
future_dict=defaultdict(int)
for penta in pentagram_table:
    token=penta[4]
    history_dict[token]+=1
    quad=penta[0:4]
    future_dict[quad]+=1
#this is to find Pcontinuation
x=len(set(pentagrams))
for penta in pentagram_table:
    key=penta[4]
    history_dict[key]=history_dict[key]/x
#print(len(history_dict))

    

N=len(pentagrams)
N1=freq_table[1]
N2=freq_table[2]
N3=freq_table[3]
N4=freq_table[4]
Y=N1/(N1+2*N2)
D=[0]*4
D[0]=Y
D[1]=1-(2*Y*(N2/N1))
D[2]=2-(3*Y*(N3/N2))
D[3]=3-(4*Y*(N4/N3))

for penta in pentagram_table:
    c=pentagram_table[penta]
    W=penta[4]
    Wh=penta[0:4]
    Pcont=history_dict[W]
    if c<=3:
        try:
            L=(D[c]/quadgram_table[Wh])*future_dict[Wh]
            Pkn=((max(c-D[c],0)/quadgram_table[Wh])+(L*Pcont))
        except:
            Pkn=0
    else:
        try:
            L=(D[3]/quadgram_table[Wh])*future_dict[Wh]
            Pkn=((max(c-D[3],0)/quadgram_table[Wh])+(L*Pcont))
        except:
            Pkn=0
    if Pkn>0.0:
        pentagram_table[penta]=Pkn
           
penta_prob_table=defaultdict(dict)
for penta in pentagram_table:
    p=pentagram_table[penta]
    quad=penta[0:4]
    token=penta[4]
    if p>0.0:
        penta_prob_table[quad][token]=p   


for penta in held_pentagramSet:
    if penta not in pentagram_table:
        c=pentagram_table[penta]
        W=penta[4]
        Wh=penta[0:4]
        Pcont=history_dict[W]
        if c<=3:
            try:
                L=(D[c]/quadgram_table[Wh])*future_dict[Wh]
                Pkn=((max(c-D[c],0)/quadgram_table[Wh])+(L*Pcont))
            except:
                pass
        else:
            try:
                L=(D[3]/quadgram_table[Wh])*future_dict[Wh]
                Pkn=((max(c-D[3],0)/quadgram_table[Wh])+(L*Pcont)) 
            except:
                pass
        if Pkn>0.0:
            penta_prob_table[penta[0:4]][penta[4]]=Pkn
            #print(Pkn)
        
for quad in penta_prob_table:
    penta_prob_table[quad]=dict(sorted(penta_prob_table[quad].items(), key=operator.itemgetter(1), reverse=True))

In [18]:
find_score(penta_prob_table,test_pentagramSet)

104336
3937


In [20]:
find_perplexity(penta_prob_table,test_pentagramSet,len(train_tokens))

1.0
